In [1]:
import pathlib
import matplotlib

import mne
import mne_bids

matplotlib.use("Qt5Agg")
mne.set_log_level('warning')

In [2]:
epochs = mne.read_epochs(fname=r'out_data/epoch_epo.fif')
epochs

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Right: 71Visual/left: 73
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [3]:
epochs.apply_baseline((None, 0))

Number of events,320
Events,Auditory/Left: 72Auditory/Right: 73Button: 16Smiley: 15Visual/Right: 71Visual/left: 73
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [4]:
epochs.plot()

<MNEBrowseFigure size 1622x986 with 4 Axes>

### 基于通道信号的幅度删除伪迹（artifacts）

In [5]:
reject_criteria  = dict(
    mag = 3000e-15, # 3000fT
    grad = 3000e-13, # 3000fT/cm
    eeg = 150e-6, # 150μV
    eog = 200e-6, # 200μV
)
flat_criteria = dict(
    mag = 1e-15, # 1fT
    grad = 1e-13, # 1fT/cm
    eeg = 1e-6, # 1μV
)

In [6]:
epochs.drop_bad(reject=reject_criteria, flat=flat_criteria)

Number of events,271
Events,Auditory/Left: 57Auditory/Right: 61Button: 15Smiley: 14Visual/Right: 57Visual/left: 67
Time range,-0.300 – 0.499 sec
Baseline,-0.300 – 0.000 sec


In [7]:
epochs.plot_drop_log()

<Figure size 1280x960 with 1 Axes>

In [8]:
epochs['Visual'].plot_image()

[<Figure size 1280x960 with 3 Axes>,
 <Figure size 1280x960 with 3 Axes>,
 <Figure size 1280x960 with 3 Axes>]

In [9]:
epochs.plot_sensors(ch_type='eeg')

<Figure size 1280x1280 with 1 Axes>

In [10]:
epochs['Visual'].plot_image(picks='EEG 058')

[<Figure size 1280x960 with 4 Axes>]

## SSP
### 使用ssp清除EOG或者ECG伪迹

In [11]:
bids_root = pathlib.Path('out_data/sample_BIDS')
bids_path = mne_bids.BIDSPath(subject='01',
                             session='01',
                             task='audiovisual',
                             run='01',
                             datatype='meg',
                             root=bids_root)
raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=0.1, h_freq=40)
ecg_projs, ecg_events = mne.preprocessing.compute_proj_ecg(raw, n_grad=1, n_mag=1, n_eeg=0, average=True)
eog_projs, eog_events = mne.preprocessing.compute_proj_eog(raw, n_grad=1, n_mag=1, n_eeg=1, average=True)


<ipython-input-11-e919b94989ea>:8: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


### 无ECG，可以通过mag建立ECG通道

In [12]:
ecg_projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | ECG-planar--0.200-0.400-PCA-01, active : False, n_channels : 203>,
 <Projection | ECG-axial--0.200-0.400-PCA-01, active : False, n_channels : 102>]

In [13]:
eog_projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>,
 <Projection | EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 59>]

In [14]:
projs = eog_projs + ecg_projs

In [15]:
projs

[<Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | EOG-planar--0.200-0.200-PCA-01, active : False, n_channels : 203>,
 <Projection | EOG-axial--0.200-0.200-PCA-01, active : False, n_channels : 102>,
 <Projection | EOG-eeg--0.200-0.200-PCA-01, active : False, n_channels : 59>,
 <Projection | PCA-v1, active : False, n_channels : 102>,
 <Projection | PCA-v2, active : False, n_channels : 102>,
 <Projection | PCA-v3, active : False, n_channels : 102>,
 <Projection | ECG-planar--0.200-0.400-PCA-01, active : False, n_channels : 203>,
 <Projection | ECG-axial--0.200-0.400-PCA-01, active : False, n_channels : 102>]

In [16]:
epochs.add_proj(projs)
epochs.plot()

<MNEBrowseFigure size 1622x986 with 5 Axes>

### 获取去除伪迹数据

In [17]:
epoch_cleaned = epochs.copy().apply_proj()
epoch_cleaned['Visual'].plot_image()
epoch_cleaned['Visual'].plot_image(picks='EEG 060')

[<Figure size 1280x960 with 4 Axes>]

## ICA
### 读取原始数据，使用1Hz的高通过滤器，这有利于ICA

In [18]:
bids_root = pathlib.Path('out_data/sample_BIDS')
bids_path = mne_bids.BIDSPath(subject='01',
                             session='01',
                             task='audiovisual',
                             run='01',
                             datatype='meg',
                             root=bids_root)
raw = mne_bids.read_raw_bids(bids_path)
raw.load_data()
raw.filter(l_freq=1.0, h_freq=40)

<ipython-input-18-29c9fa4a9229>:8: RuntimeWarning: The unit for channel(s) STI 001, STI 002, STI 003, STI 004, STI 005, STI 006, STI 014, STI 015, STI 016 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path)


<Raw | sub-01_ses-01_task-audiovisual_run-01_meg.fif, 376 x 166800 (277.7 s), ~481.8 MB, data loaded>

In [19]:
raw.plot()

<MNEBrowseFigure size 1622x986 with 5 Axes>

In [20]:
epochs = mne.read_epochs(fname=r'out_data/epoch_epo.fif')
epochs_selection = epochs.selection
epochs_selection

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

### 用于ICA的epoch和最开始创建的epoch不一样
### 所以为了ICA,需要重新创建epoch

In [21]:
events, events_id = mne.events_from_annotations(raw)
events = events[epochs_selection]

In [22]:
tmin = epochs.tmin
tmax = epochs.tmax

baseline = (None, 0)

epochs_ica = mne.Epochs(raw,
                       events=events,
                       event_id=events_id,
                       tmin=tmin,
                       tmax=tmax,
                       baseline=baseline,
                       preload=True)


### 用ICA拟合数据

In [28]:
n_components = 0.8
method = 'picard'
max_iter = 100 #实际应该视收敛情况而定
fit_params = dict(fastica_it=5)
random_state = 42
ica = mne.preprocessing.ICA(n_components=n_components, 
                           #max_pca_components=300,
                           method=method,
                           max_iter=max_iter,
                           fit_params=fit_params,
                           random_state=random_state)
ica.fit(epochs_ica)

<ipython-input-28-ee44254c70f0>:12: RuntimeWarning: The epochs you passed to ICA.fit() were baseline-corrected. However, we suggest to fit ICA only on data that has been high-pass filtered, but NOT baseline-corrected.
  ica.fit(epochs_ica)
d:\anaconda\envs\tf2.1\lib\site-packages\picard\solver.py:216: UserWarning: Picard did not converge. Final gradient norm : 4.683e-06. Requested tolerance : 1e-07. Consider increasing the number of iterations or the tolerance.
  % (gradient_norm, tol))


<ICA | epochs decomposition, fit (picard): 153920 samples, 28 components, channels used: "mag"; "grad"; "eeg">

In [29]:
ica.plot_components(inst=epochs)

[<MNEFigure size 1920x986 with 20 Axes>, <MNEFigure size 1924x992 with 8 Axes>]

### 检测ecg、eog相关的伪迹

In [30]:
ecg_epochs = mne.preprocessing.create_ecg_epochs(raw, reject=None, 
                                                baseline=(None, -0.2), 
                                                tmin=-0.5, tmax=0.5)
ecg_evoked = ecg_epochs.average()
ecg_inds, ecg_scores = ica.find_bads_ecg(
                    ecg_epochs, method='ctps'
)

eog_epochs = mne.preprocessing.create_eog_epochs(raw, reject=None, 
                                                baseline=(None, -0.2), 
                                                tmin=-0.5, tmax=0.5)
eog_evoked = ecg_epochs.average()
eog_inds, eog_scores = ica.find_bads_eog(
                    eog_epochs
)
components_to_exclude = ecg_inds + ecg_inds
ica.exclude = components_to_exclude

### eog、ecg伪迹决策的分数

In [32]:
ica.plot_scores(ecg_scores)

<Figure size 1280x540 with 1 Axes>

In [33]:
ica.plot_scores(eog_scores)

<Figure size 1280x540 with 1 Axes>

### ica的分数

In [37]:
ica.plot_sources(ecg_evoked)

<Figure size 1280x960 with 1 Axes>

In [38]:
ica.plot_sources(eog_evoked)

<Figure size 1280x960 with 1 Axes>

In [40]:
ica.exclude

[0, 14, 0, 14]

### 画出overlay的原始和清理过后的数据

In [41]:
ica.plot_overlay(ecg_evoked)

<ipython-input-41-4cf3391a4078>:1: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  ica.plot_overlay(ecg_evoked)


<Figure size 1280x960 with 3 Axes>

In [42]:
ica.plot_overlay(eog_evoked)

<ipython-input-42-434730e1d117>:1: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  ica.plot_overlay(eog_evoked)


<Figure size 1280x960 with 3 Axes>

<div class="alert alert-info" role="alert">生成ica需要使用特定过滤器过滤过的数据，最后应用的时候则不是</div>


In [45]:
epochs_cleaned = ica.apply(epochs.copy())

<ipython-input-45-fa7578ef095d>:1: RuntimeWarning: The data you passed to ICA.apply() was baseline-corrected. Please note that ICA can introduce DC shifts, therefore you may wish to consider baseline-correcting the cleaned data again.
  epochs_cleaned = ica.apply(epochs.copy())


In [46]:
epochs_cleaned.plot()

<MNEBrowseFigure size 1622x986 with 4 Axes>

In [ ]:
epochs_cleaned.save('')